In [29]:
import pinecone
from tqdm.autonotebook import tqdm
import os
# Set your Pinecone API key
api_key = os.getenv("PINECONE_API_KEY")
pinecone.init(api_key=api_key, environment="us-central1-gcp")


# Get the index 
pinecone.list_indexes()


pinecone.describe_index("icold")
index_name = "icold"
index = pinecone.GRPCIndex(index_name)

index.describe_index_stats()
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=os.getenv("OPENAI_API_KEY"),
)
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)
query = "what is a cfrd?"

vectorstore.similarity_search(
    query,  # our search query
    k=5  # return 3 most relevant docs
)
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# completion llm
llm = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
qa.run(query)
from langchain.chains import RetrievalQAWithSourcesChain

qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)
response = qa_with_sources(query)


'G:\\.shortcut-targets-by-id\\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\\ICOLD - CFRD New Bulletin 2023\\Dam Response\\jgeot.17.p.095.pdf, G:\\.shortcut-targets-by-id\\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\\ICOLD - CFRD New Bulletin 2023\\Structural Analysis\\Behaviour of the reinforced concrete face slabs of concrete faced rockfill dams during impounding.pdf, G:\\.shortcut-targets-by-id\\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\\ICOLD - CFRD New Bulletin 2023\\Dam Response\\jgeot.17.p.095.pdf, G:\\.shortcut-targets-by-id\\1vE28d8xZuJXkpcinFbuku9FJgeaDd48K\\ICOLD - CFRD New Bulletin 2023\\Numerical Analysis\\Fu2019_Article_AGeneralizedPlasticityModelFor.pdf'